# Model Evaluation :: Data Preparation

As a part of the generalized evaluation workflow: 

<img src='./Eval_PreProc.svg' width=600>

The pre-processing step is needed in order to align the two datasets for analysis.  The specific 
steps needed to prepare a given dataset may differ, depending on the source and the variable of
interest. 

Some steps might include: 

* Organizing the time-series index such that the time steps for both simulated and observed are congruent
    * This may involve interpolation to estimate a more granular time-step than is found in the source data
    * More often, an agregating function is used to 'down-sample' the dataset to a coarser time step (days vs hours).
* Coordinate aggregation units between simulated and observed 
    * Gridded data may be sampled per HUC-12, HUC-6, etc. to match modeled data indexed by these units. 
    * Index formats may be adjusted (e.g. a 'gage_id' may be 'USGS-01104200' in one data set, vs '01104200' in another)
* Re-Chunking the data to make time-series analysis more efficient (see [here](/dev/null) for a primer on re-chunking).

## Streamflow Data Prep

This document shows one approach to preparing the _streamflow_ data for subsequent analysis (That analysis is outlined [here](./02_Analysis_NWM.ipynb)).

Streamflow analysis will compare time-series of two aligned datasets: 
* 'observed' data values obtained from [NWIS](https://nwis.waterdata.usgs.gov/nwis) 
* 'modeled' data extracted from the [NWM](https://registry.opendata.aws/nwm-archive/)

These data soruces are accessed using different methods.  We will pull data from their respective sources, reshape and optimize the data structures, then write that data to storage to make later analysis easier. 

## Sourcing the 'modeled' Data
Modeled data for this demonstration tutorial will be sourced from the S3 bucket `nhgf-development`.

In [1]:
import os
import fsspec
import dask
import xarray as xr
os.environ['AWS_PROFILE'] = 'osn-renci'
fs_write = fsspec.filesystem('s3', 
                         profile='osn-renci', 
                         skip_instance_cache=True, 
                         client_kwargs={'endpoint_url': 'https://renc.osn.xsede.org'}
                        )
# os.environ['AWS_PROFILE'] = 'nhgf-development'  # credentials are required
fs_read = fsspec.filesystem('s3', profile='nhgf-development') # create reference to a S3 filesystem driver.

modeled = xr.open_zarr(
    fs_read.get_mapper('s3://nhgf-development/nwm/chanobs.zarr'),  #<<< example data source of NWM2.1  CHANOBS_DOMAIN
    consolidated=False, 
    chunks={}, 
    drop_variables='velocity' #<< not using this var; just 'streamflow'
)
modeled

<xarray.Dataset>
Dimensions:     (gage_id: 7994, feature_id: 7994, time: 367439)
Coordinates:
    elevation   (gage_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
  * feature_id  (feature_id) int32 3109 3923 12932 ... 1170023539 1180000535
  * gage_id     (gage_id) <U20 'USGS-01030350' ... 'USGS-10254970'
    latitude    (gage_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
    longitude   (gage_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
    order       (gage_id) int32 dask.array<chunksize=(7994,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Data variables:
    streamflow  (time, gage_id) float64 dask.array<chunksize=(367439, 1), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...


**Source Data as Template**

This source data set will establish the indices and boundaries for the data we will eventually pull from the NWIS stream gage network.
The two dimensions of this data are the **Gage ID** and **Time**.  We'll use these dimensions to fetch the 'observed' data later. The 
endpoints and range of these dimensions will establish that future query.    

In [2]:
## Gage IDs
import re
_gages = [gage_id.lstrip() for gage_id in modeled['gage_id'].values.astype('str')]
GAGES = [g for g in _gages if re.search('USGS-\d+$', g) ]  # note the regex search pattern

# Time bounds
start_time = modeled.time.values.min()
stop_time = modeled.time.values.max()
DATE_RANGE=(start_time, stop_time)

Some important notes for each of those bounds: 
* **GAGES** / Gage IDs :: The list of 7994 gage IDs in the model dataset include some values which the NWIS does not 
  recognize and will not accept. We need to remove them. 
  * Gage IDs of the form `USGS-\d+` (A string starting 'USGS-' and ending in an arbitrary number of digits)
    are processed by NWIS data requests.
  * There are roughly 350 gage IDs in the modeled dataset with letters embedded in the string of digits after the 'USGS-'. These
    will be rejecte when we try to call NWIS data service to obtain streamflow history for that location.
  * This is the reason behind the regular expression search (`re.search`) to select only gage_id of the correct format. 
  * After selecting the NWIS-compliant gage IDs, the `GAGES` list contains 7647 gages.
* **DATE_RANGE** / Dates :: This defines the temporal range for the historical data will will fetch from NWIS. 
  * The NWM modeled data includes time values stepped hourly.
  * The historical streamflow data is stepped daily.
  * We will resample later to make sure the indices match. 

In [3]:
GAGES[0:10]

['USGS-01030350',
 'USGS-01030500',
 'USGS-06741510',
 'USGS-402114105350101',
 'USGS-272524080221800',
 'USGS-02277600',
 'USGS-254157080213800',
 'USGS-022907085',
 'USGS-022908295',
 'USGS-06711780']

## Sourcing the 'observed' data

Now that we have information about the list of gages and the date range covered by the model, we 
can use that to query NWIS for matching data points for this same range of dates and station IDs. 
Because NWIS data is structured a little differently than the modeled streamflow, we'll need to 
re-arrange the data a little after fetching. 

In addition, a call to NWIS for historical data can be time consuming -- and we will do it roughly 
7500 times.  We will eventually set up a mechanism to do these requests in parallel, once we've 
established how the data restructuring should happen. 

The first step in that process is to make a NWIS request for just a couple of gages to see how 
the return data is structured.  We'll use that information to create the plan by which the full 
dataset is to be fetched and reorganized. 

In [4]:
from pygeohydro import NWIS
nwis = NWIS()
## Fetch data for a couple of gages to see how NWIS formats a response
observed = nwis.get_streamflow(GAGES[0:2], DATE_RANGE, to_xarray=True) 
observed

<xarray.Dataset>
Dimensions:       (time: 15310, station_id: 2)
Coordinates:
  * time          (time) datetime64[ns] 1979-02-01T05:00:00 ... 2020-12-31T05...
  * station_id    (station_id) object 'USGS-01030350' 'USGS-01030500'
Data variables:
    discharge     (time, station_id) float64 nan 116.1 nan ... 193.7 2.093 158.3
    station_nm    (station_id) <U43 'Wytopitlock Stream Near Wytopitlock, Mai...
    dec_lat_va    (station_id) float64 45.71 45.5
    dec_long_va   (station_id) float64 -68.16 -68.31
    alt_va        (station_id) float64 395.0 217.0
    alt_acy_va    (station_id) int64 5 10
    alt_datum_cd  (station_id) <U6 'NGVD29' 'NGVD29'
    huc_cd        (station_id) <U8 '01020003' '01020003'
    begin_date    (station_id) <U10 '2008-10-01' '1934-10-01'
    end_date      (station_id) <U10 '2022-10-10' '2022-10-10'
Attributes:
    tz:       UTC

## Re-arrange the response data
We requested two stream gages rather than just one, to ensure that the dataset is multi-dimensional 
(as the final request will be). 

Now that we can see the data provided by `NWIS`, we can begin to shape the plan to match its 
structure to the modeled data. 

**Rename Vars** -- First step is to adjust some variable names in order to match the
modeled data above ('_discharge_' --> '_streamflow_', etc).  

In [5]:
observed = (observed
            .rename_dims({'station_id':'gage_id'})
            .rename({'discharge':'streamflow', 'station_id':'gage_id'})
           )
observed


/tmp/ipykernel_65/3019346767.py:1: UserWarning: rename 'station_id' to 'gage_id' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  observed = (observed


<xarray.Dataset>
Dimensions:       (time: 15310, gage_id: 2)
Coordinates:
  * time          (time) datetime64[ns] 1979-02-01T05:00:00 ... 2020-12-31T05...
  * gage_id       (gage_id) object 'USGS-01030350' 'USGS-01030500'
Data variables:
    streamflow    (time, gage_id) float64 nan 116.1 nan ... 193.7 2.093 158.3
    station_nm    (gage_id) <U43 'Wytopitlock Stream Near Wytopitlock, Maine'...
    dec_lat_va    (gage_id) float64 45.71 45.5
    dec_long_va   (gage_id) float64 -68.16 -68.31
    alt_va        (gage_id) float64 395.0 217.0
    alt_acy_va    (gage_id) int64 5 10
    alt_datum_cd  (gage_id) <U6 'NGVD29' 'NGVD29'
    huc_cd        (gage_id) <U8 '01020003' '01020003'
    begin_date    (gage_id) <U10 '2008-10-01' '1934-10-01'
    end_date      (gage_id) <U10 '2022-10-10' '2022-10-10'
Attributes:
    tz:       UTC


**Chunking**  -- The next step in organizing this data is to define how it will be 'chunked' when written to storage. 

:::{sidebar}
See [here](/dev/null) for a primer in chunking.
:::

We'll do this by creating an 'empty' dataset using our sample of observed values as a template, with the
chunking plan laid out as we like. Once we have the template sorted out, we can then populate the template
with  data fetched from NWIS for all of the gages of interest. 

In [6]:
source_dataset = observed
template = (xr.zeros_like(source_dataset)  # DataSet just like 'observed'
             .chunk()           
             .isel(gage_id=0, drop=True)      # temporarily remove gage_id as a dimension and coordinate
             .expand_dims(gage_id=len(GAGES), axis=-1) # add it back, reserving space for the full size of GAGES
             .assign_coords({'gage_id': GAGES}) # add coordinate to match dimension
             .chunk({                         # define chunk sizes
                 'time': len(observed.time),  # all time vals in one chunk
                 'gage_id': 1}                # one gage_id per chunk
             )
           )
template

<xarray.Dataset>
Dimensions:       (time: 15310, gage_id: 7647)
Coordinates:
  * time          (time) datetime64[ns] 1979-02-01T05:00:00 ... 2020-12-31T05...
  * gage_id       (gage_id) <U20 'USGS-01030350' ... 'USGS-10254970'
Data variables:
    streamflow    (time, gage_id) float64 dask.array<chunksize=(15310, 1), meta=np.ndarray>
    station_nm    (gage_id) <U43 dask.array<chunksize=(1,), meta=np.ndarray>
    dec_lat_va    (gage_id) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    dec_long_va   (gage_id) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    alt_va        (gage_id) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    alt_acy_va    (gage_id) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    alt_datum_cd  (gage_id) <U6 dask.array<chunksize=(1,), meta=np.ndarray>
    huc_cd        (gage_id) <U8 dask.array<chunksize=(1,), meta=np.ndarray>
    begin_date    (gage_id) <U10 dask.array<chunksize=(1,), meta=np.ndarray>
    end_date      (gage_id) <U10 dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    tz:       UTC

**Formalize Template** -- We can now write this empty template to storage, which will codify the 
metadata and create a landing zone for the upcoming parallelized calls to the NWIS service. 

In [30]:
fname='s3://rsignellbucket2/testing/gzt5142/nwis_out_gzt5142.zarr'
if fs_write.exists(fname):
    fs_write.rm(fname, recursive=True)

In [11]:
import numpy as np

outfile = fs_write.get_mapper(fname)
template.to_zarr(
    outfile,
    compute=False,                                 # << will not write data; just metadata
    encoding =  {                                  # encodings sets data types for the disk store
        'station_nm':  dict( _FillValue='',          dtype='<U64'), 
        'alt_datum_cd': dict(_FillValue='',          dtype('<U6'),
        'alt_acy_va':  dict( _FillValue=-2147483647, dtype=np.int32),
        'alt_va':      dict( _FillValue=9.96921e+36, dtype=np.float32),
        'dec_lat_va':  dict( _FillValue=None,        dtype=np.float32),
        'dec_long_va': dict( _FillValue=None,        dtype=np.float32),
        'streamflow':  dict( _FillValue=9.96921e+36, dtype=np.float32)
    },
    consolidated=True,                             # Consolidate metadata
    mode='w'
)

Delayed('_finalize_store-7d8dabfd-b0d2-4e8e-9034-691970fcda33')

## Fetch All NWIS Data
Now that we have established the names, encoding, chunking, etc of the output data, we can now
fetch the streamflow data froM NWIS and populate the data store.  This will involve over 7000
calls to the NWIS service (one per gage), so we will do that in parallel. 

To set up for parallel, we will establish an encapuslated routine to process a single stream
gage.  We will then farm out that routine to workers -- a worker will process a single gage
from the list. 

In [12]:
# Global: 
n_timesteps = len(observed.time)
time_steps = observed.time.values
# observed.to_zarr(outfile,  region={'time':slice(0, n_timesteps), 'gage_id':slice(0, 2)})

def write_one_gage(n):
    """ 
    Writes one gage's data to the existing zarr file.  Uses the NWIS API call to fetch data.
    
    Arguments: 
    n   : integer
       the index into the GAGES array identifying which gage to fetch and write. 
    """
    site_id = GAGES[n]
    try:
        _obs = nwis.get_streamflow(site_id, DATE_RANGE, to_xarray=True).interp(time=time_steps)
        _obs = _obs.rename_dims({'station_id':'gage_id'}).rename({'station_id':'gage_id','discharge':'streamflow'})
        ## We must force the returned data into the datatype that we stored to disk. 
        _obs['station_nm'] = xr.DataArray(data=_obs['station_nm'].values.astype('<U32'), dims='gage_id')
        _obs['alt_datum_cd'] = xr.DataArray(data=_obs['alt_datum_cd'].values.astype('<U6'), dims='gage_id')
 
        _obs.to_zarr(
            outfile, 
            region={ #<<< Specifying a region lets us 'insert' data to a specific place in the dataset. 
                'time': slice(0, n_timesteps), 
                'gage_id': slice(n,n+1)
                }
            )
        return n
    except: # This is an extremely broad way to catch exceptions... and in general is to be avoided. 
            # We do it this way in this case to protect the parallel run. it allows a single write_one_gage() 
            # to fail silently without affecting the rest of the run.
        return None

In [13]:
# test to see if it will write data for one gage... pick a random item from the GAGES list
write_one_gage(3)

/tmp/ipykernel_79/383510806.py:17: UserWarning: rename 'station_id' to 'gage_id' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  _obs = _obs.rename_dims({'station_id':'gage_id'}).rename({'station_id':'gage_id','discharge':'streamflow'})


3

**Start Cluster** -- Finally, we are ready to run the above routine in a clustered compute environment.
As this routine will write to an S3 bucket, the cluster should be configured with the **SAME CREDENTIALS**
as the bucket access profile. 

In [8]:
# This cluster configuration is VERY specific to the JupyterHub cloud environment on ESIP/QHUB
from dask.distributed import as_completed

import sys
sys.path.append('/shared/users/lib')
try:
    import ebdpy as ebd
except ImportError as exc:
    raise ImportError("EBD library not found!!") from exc

#ebd.set_credentials(profile='osn-renci')
aws_profile = 'osn-renci'
aws_region = 'us-east-1'
endpoint = f'https://renc.osn.xsede.org'
ebd.set_credentials(profile=aws_profile, region=aws_region, endpoint=endpoint)
worker_max = 30
client, cluster = ebd.start_dask_cluster(
    profile=aws_profile,
    worker_max=worker_max,
    region=aws_region,
    use_existing_cluster=True,
    adaptive_scaling=False,
    wait_for_cluster=False,
    worker_profile='Medium Worker',
    propagate_env=True
)

Region: us-east-1
Existing Dask clusters:
Cluster Index c_idx: 0 / Name: dev.3a1612dc6dab42a0a283354d08e1779c ClusterStatus.RUNNING
Using existing cluster [0].
Setting Fixed Scaling workers=30
Reconnect client to clear cache
client.dashboard_link (for new browser tab/window or dashboard searchbar in Jupyterhub):
https://jupyter.qhub.esipfed.org/gateway/clusters/dev.3a1612dc6dab42a0a283354d08e1779c/status
Propagating environment variables to workers
Using environment: users/pangeo


**RUN** -- We have a few options for parallelizing this task.  Simplest is to use a dask `bag`:

In [ ]:
%%time
import dask.bag as db
b = db.from_sequence(range(len(GAGES)), npartitions=1000)
b = b.map(write_one_gage)

_ = b.compute()

In [17]:
## Done with parallel work... shut down the cluster
client.close()
cluster.close()

/home/conda/users/26482e3a48d85ce9bb19209dc4bfa71cb97e88608cd08cccdc341be4663d6b66-20221012-150047-575326-183-pangeo/lib/python3.9/site-packages/dask_gateway/client.py:1014: RuntimeWarning: coroutine 'rpc.close_rpc' was never awaited
  self.scheduler_comm.close_rpc()


## Verify

At this point, the data should be written to the `outfile`. We can open it with a standard `open_dataset`
using the _zarr_ data engine.  Let's do that to see what we got: 

In [18]:
dst = xr.open_dataset(outfile, engine='zarr', chunks={}, backend_kwargs=dict(consolidated=True))
dst

<xarray.Dataset>
Dimensions:       (gage_id: 7647, time: 15310)
Coordinates:
  * gage_id       (gage_id) <U20 'USGS-01030350' ... 'USGS-10254970'
  * time          (time) datetime64[ns] 1979-02-01T05:00:00 ... 2020-12-31T05...
Data variables:
    alt_acy_va    (gage_id) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    alt_datum_cd  (gage_id) <U6 dask.array<chunksize=(1,), meta=np.ndarray>
    alt_va        (gage_id) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    begin_date    (gage_id) <U10 dask.array<chunksize=(1,), meta=np.ndarray>
    dec_lat_va    (gage_id) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    dec_long_va   (gage_id) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    end_date      (gage_id) <U10 dask.array<chunksize=(1,), meta=np.ndarray>
    huc_cd        (gage_id) <U8 dask.array<chunksize=(1,), meta=np.ndarray>
    station_nm    (gage_id) object dask.array<chunksize=(1,), meta=np.ndarray>
    streamflow    (time, gage_id) float32 dask.array<chunksize=(15310, 1), meta=np.ndarray>
Attributes:
    tz:       UTC

In [16]:
n_timesteps = len(observed.time)
time_steps = observed.time.values
def fetch_gages(site_IDs):
    """ 
    Writes one gage's data to the existing zarr file.  Uses the NWIS API call to fetch data.
    
    Arguments: 
    site_IDs: iterable with SiteID strings 
    """

    try:
        _obs = nwis.get_streamflow(site_IDs, DATE_RANGE, to_xarray=True).interp(time=time_steps)
        _obs = _obs.rename_dims({'station_id':'gage_id'}).rename({'station_id':'gage_id','discharge':'streamflow'})
        ## We must force the returned data into the datatype that we stored to disk. 
        #_obs['station_nm'] = xr.DataArray(data=_obs['station_nm'].values.astype('<U32'), dims='gage_id')
        #_obs['alt_datum_cd'] = xr.DataArray(data=_obs['alt_datum_cd'].values.astype('<U6'), dims='gage_id')
        return _obs
    except:
        raise
        # This is an extremely broad way to catch exceptions... and in general is to be avoided. 
            # We do it this way in this case to protect the parallel run. it allows a single write_one_gage() 
            # to fail silently without affecting the rest of the run.
        #return None

In [21]:
## Dispatch requests to the NWIS server in batches of `stepsize` gages. 
remaining = len(GAGES)
stepsize = 500
batch=0
futures=[]
while remaining > 0:
    if remaining < stepsize:
        n = remaining
    else:
        n = stepsize
    start_i = batch * stepsize
    end_i = (batch * stepsize) + n
    futures.append(client.submit(fetch_gages, GAGES[start_i:end_i]))
    batch += 1
    remaining -= n


In [22]:
%%time
firstreturn=True
for (f, r) in as_completed(futures, with_results=True):
    if firstreturn:
        firstreturn=False
        accumulator = xr.merge([r])
    else:
        print(".", end='')
        accumulator = xr.merge([accumulator, r])
print("done")
del futures

...............done
CPU times: user 33.3 s, sys: 8.94 s, total: 42.2 s
Wall time: 2min 34s


In [23]:
accumulator

<xarray.Dataset>
Dimensions:       (gage_id: 7489, time: 15310)
Coordinates:
  * gage_id       (gage_id) object 'USGS-01010000' ... 'USGS-480608115242901'
  * time          (time) datetime64[ns] 1979-02-01T05:00:00 ... 2020-12-31T05...
Data variables:
    streamflow    (time, gage_id) float64 39.64 nan 86.37 ... 1.144 nan 0.0201
    station_nm    (gage_id) object 'St. John River At Ninemile Bridge, Maine'...
    dec_lat_va    (gage_id) float64 46.7 46.89 47.11 47.07 ... 41.41 44.27 48.1
    dec_long_va   (gage_id) float64 -69.72 -69.75 -69.09 ... -88.08 -115.4
    alt_va        (gage_id) float64 931.3 885.0 590.4 ... 628.4 900.0 3.02e+03
    alt_acy_va    (gage_id) float64 0.01 20.0 0.01 0.01 ... 0.1 0.1 5.0 40.0
    alt_datum_cd  (gage_id) object 'NGVD29' 'NGVD29' ... 'NAVD88' 'NGVD29'
    huc_cd        (gage_id) object '01010001' '01010001' ... '17010102'
    begin_date    (gage_id) object '1950-10-01' '1983-10-01' ... '2011-10-01'
    end_date      (gage_id) object '2022-10-17' '2022-10-17' ... '2022-07-04'
Attributes:
    tz:       UTC

In [27]:
output=accumulator.chunk({                         # define chunk sizes
                 'time': len(accumulator.time),  # all time vals in one chunk
                 'gage_id': 1}                # one gage_id per chunk
             )

In [28]:
output

<xarray.Dataset>
Dimensions:       (gage_id: 7489, time: 15310)
Coordinates:
  * gage_id       (gage_id) object 'USGS-01010000' ... 'USGS-480608115242901'
  * time          (time) datetime64[ns] 1979-02-01T05:00:00 ... 2020-12-31T05...
Data variables:
    streamflow    (time, gage_id) float64 dask.array<chunksize=(15310, 1), meta=np.ndarray>
    station_nm    (gage_id) object dask.array<chunksize=(1,), meta=np.ndarray>
    dec_lat_va    (gage_id) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    dec_long_va   (gage_id) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    alt_va        (gage_id) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    alt_acy_va    (gage_id) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    alt_datum_cd  (gage_id) object dask.array<chunksize=(1,), meta=np.ndarray>
    huc_cd        (gage_id) object dask.array<chunksize=(1,), meta=np.ndarray>
    begin_date    (gage_id) object dask.array<chunksize=(1,), meta=np.ndarray>
    end_date      (gage_id) object dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    tz:       UTC

In [31]:
%%time
import numpy as np
fname='s3://rsignellbucket2/testing/gzt5142/nwis_out_gzt5142.zarr'
if fs_write.exists(fname):
    print("Removing old copy of outout...", end="")
    fs_write.rm(fname, recursive=True)
    print("Done")
    
outfile = fs_write.get_mapper(fname)
output.to_zarr(
    outfile,
    encoding =  {                                  # encodings sets data types for the disk store
        'station_nm':  dict( _FillValue='',          dtype='<U64'), 
        'alt_datum_cd': dict(_FillValue='',          dtype='<U6'),
        'alt_acy_va':  dict( _FillValue=-2147483647, dtype=np.int32),
        'alt_va':      dict( _FillValue=9.96921e+36, dtype=np.float32),
        'dec_lat_va':  dict( _FillValue=None,        dtype=np.float32),
        'dec_long_va': dict( _FillValue=None,        dtype=np.float32),
        'streamflow':  dict( _FillValue=9.96921e+36, dtype=np.float32)
    },
    consolidated=True,                             # Consolidate metadata
    mode='w'
)

/home/conda/users/7f010de3e770efba6e7f48576350290478de8e7b231142948c80368e2fb75908-20221017-193806-084368-195-pangeo/lib/python3.9/site-packages/xarray/conventions.py:205: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(
/home/conda/users/7f010de3e770efba6e7f48576350290478de8e7b231142948c80368e2fb75908-20221017-193806-084368-195-pangeo/lib/python3.9/site-packages/xarray/conventions.py:205: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(


CancelledError: ('store-map-242f85c476fc5e90b1cb66258b95b57b', 1509)

2022-10-18 08:58:25,614 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [ ]:
dst = xr.open_dataset(outfile, engine='zarr', chunks={}, backend_kwargs=dict(consolidated=True))
dst